# Load Klimadaten

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from pathlib import Path

In [3]:
url = 'https://climateactiontracker.org/countries/'
website = requests.get(url)
soup = BeautifulSoup(website.content, 'html.parser')

In [4]:
div = soup.find('div', {"data-component-map": True})

# Export from Drom
df_mapdata = pd.DataFrame(json.loads(div.attrs['data-props-mapdata']))
df_ratingsdata = pd.DataFrame(json.loads(div.attrs['data-props-ratingsdata']))

# Rename
df_ratingsdata.rename(columns={'name': 'rating_name', 'description': 'rating_description'}, inplace=True)

# Merge
df = df_mapdata.merge(df_ratingsdata, left_on = 'rating', right_on = 'weight', how='left')

# Save as CSV
df.to_csv(Path('../export/data.csv'), index=False)

## Export as GeoJSON

In [21]:
# Read Geojson
with open(Path('../data/world.geojson'), 'r') as f:
    geo = json.loads(f.read())

for feature in geo['features']:
    df_c = df[df.country_code == feature['properties']['iso_a3']]
    if len(df_c) > 0:
        if df_c.iloc[0]['weight'] == 0:
            c = '#c3c2ca'
            l = "Kein Rating"
        elif df_c.iloc[0]['weight'] == 1:
            c = '#d64b47'
            l = "Kritisch ungenügend"
        elif df_c.iloc[0]['weight'] == 2:
            c = '#d96f4e'
            l = "Höchst ungenügend"
        elif df_c.iloc[0]['weight'] == 3:
            c = '#da8d5f'
            l = "Ungenügend"
        elif df_c.iloc[0]['weight'] == 4:
            c = '#ddc197'
            l = "2-Grad-Ziel wird erreicht"
        elif df_c.iloc[0]['weight'] == 5:
            c = '#919cf2'
            l = "1,5-Grad-Ziel wird erreicht"
        else:
            c = '#d4d6dd'
    else:
        c = '#d4d6dd'
        l = "Keine Daten"

    feature['properties']['fill'] = c
    feature['properties']['fill-opacity'] = 1
    feature['properties']['stroke'] = '#ffffff'
    feature['properties']['stroke-width'] = 2
    feature['properties']['stroke-opacity'] = 1
    feature['properties']['label'] = l


with (open(Path('../export/export.geojson'), 'w')) as f:
    f.write(json.dumps(geo))